# **Frame Scorer Benchmark**

## **Strategies Tested**
1. **CLIP Scoring** - Question-Frame Similarity using OpenAI CLIP
   - ViT-L/14 (best quality)
   - ViT-B/32 (faster)
2. **Multilingual CLIP** - Direct Vietnamese support
3. **Detection Scoring** - YOLO-based object detection boost
4. **Distinctiveness** - Inter-Frame Distinctiveness (IFD)
5. **Combined Scoring** - Weighted combination ($\alpha$ $\cdot$ QFS + $\beta$ $\cdot$ DGS + $\gamma$ $\cdot$ IFD)

## **Metrics Tracked**
- Model initialization time
- Inference time per frame
- Memory usage
- Score quality/distribution

## **Setup**

In [1]:
import sys
import os
import time
import gc
import psutil
import warnings
from pathlib import Path
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

sys.path.insert(0, '..')

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

from src.perception.frame_scorer import (
    FrameScorer,
    ScoringConfig,
    FrameScore,
    CLIPScoringStrategy,
    MultilingualCLIPScoringStrategy,
    DetectionScoringStrategy,
    DistinctivenessStrategy,
    get_available_strategies,
    create_scorer,
)

from config.settings import get_path_config
from src.perception.query_analyzer import QueryAnalyzer
from src.perception.model_registry import ModelRegistry, get_model

print(f"Available strategies: {get_available_strategies()}")
print(f"Available YOLO models: {ModelRegistry.list_models()}")

Available strategies: ['clip', 'mclip', 'detection', 'combined']
Available YOLO models: ['yolo11n_road_lane', 'yolo11n_bdd100k', 'yolo11l_road_lane', 'yolo11l_bdd100k', 'yolo11n_unified', 'yolo11l_unified']


## **Data Preparation**

In [2]:
paths = get_path_config()
train_videos_dir = paths.train_videos_dir
video_files = list(train_videos_dir.glob("*.mp4"))[:3]

TEST_VIDEO = str(video_files[0])
print(f"Using test video: {Path(TEST_VIDEO).name}")

Using test video: 001a9a8b_340_clip_006_0037_0046_Y.mp4


In [3]:
def extract_frames(video_path: str, num_frames: int = 16) -> List[np.ndarray]:
    """Extract frames uniformly from video."""
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Sample uniformly
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    
    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
    
    cap.release()
    return frames

# Extract test frames
TEST_FRAMES = extract_frames(TEST_VIDEO, num_frames=16)
print(f"Extracted {len(TEST_FRAMES)} frames")
print(f"Frame shape: {TEST_FRAMES[0].shape}")

Extracted 16 frames
Frame shape: (1440, 2560, 3)


In [4]:
# Test questions (Vietnamese)
TEST_QUESTIONS = [
    "Biển báo tốc độ tối đa là bao nhiêu?",
    "Có đèn đỏ trong video không?",
    "Xe có được phép rẽ trái không?",
    "Có bao nhiêu làn đường?",
    "Biển cấm nào xuất hiện trong video?",
]

# Use query analyzer to get target objects
analyzer = QueryAnalyzer(strategy="keyword")

print("Test Questions with Target Objects:")

for i, q in enumerate(TEST_QUESTIONS):
    result = analyzer.analyze(q)
    print(f"\n{i+1}. {q}")
    print(f"   Target Objects: {result.target_objects[:3]}")
    print(f"   YOLO Classes: {result.yolo_classes}")

Test Questions with Target Objects:

1. Biển báo tốc độ tối đa là bao nhiêu?
   Target Objects: ['traffic_sign', 'max_speed_sign', 'speed_limit_sign']
   YOLO Classes: ['traffic sign']

2. Có đèn đỏ trong video không?
   Target Objects: ['red_light', 'traffic_light_red']
   YOLO Classes: ['traffic light']

3. Xe có được phép rẽ trái không?
   Target Objects: ['left_turn', 'direction_left']
   YOLO Classes: ['traffic sign']

4. Có bao nhiêu làn đường?
   Target Objects: ['lane', 'lane_marking']
   YOLO Classes: ['road']

5. Biển cấm nào xuất hiện trong video?
   Target Objects: ['prohibitory_sign']
   YOLO Classes: ['traffic sign']


## **Benchmark Utilities**

In [5]:
@dataclass
class BenchmarkResult:
    """Result from benchmarking a scorer."""
    strategy_name: str
    model_name: str = ""
    init_time_ms: float = 0.0
    avg_inference_ms: float = 0.0
    total_inference_ms: float = 0.0
    memory_mb: float = 0.0
    avg_score: float = 0.0
    score_std: float = 0.0
    min_score: float = 0.0
    max_score: float = 0.0
    scores: np.ndarray = field(default_factory=lambda: np.array([]))
    success: bool = True
    error: str = ""


def get_memory_mb() -> float:
    """Get current process memory in MB."""
    return psutil.Process().memory_info().rss / (1024 * 1024)

all_results: List[BenchmarkResult] = []

## 1. **CLIP Scoring Strategy Benchmark**

Testing OpenAI CLIP models for Question-Frame Similarity (QFS).

In [6]:
# CLIP models to test
CLIP_MODELS = [
    "ViT-B/32",
    "ViT-L/14",
]

clip_results = []
test_question = TEST_QUESTIONS[0]

for model_name in CLIP_MODELS:
    print(f"Testing CLIP model: {model_name}")
    
    gc.collect()
    mem_before = get_memory_mb()
    
    try:
        # Initialize
        init_start = time.perf_counter()
        scorer = CLIPScoringStrategy(
            model_name=model_name,
            device="cpu",
            use_translation=True
        )
        init_time = (time.perf_counter() - init_start) * 1000
        
        mem_after = get_memory_mb()
        memory_used = mem_after - mem_before
        
        print(f"  Init time: {init_time:.2f} ms")
        print(f"  Memory: {memory_used:.1f} MB")
        
        # Warmup
        _ = scorer.score(TEST_FRAMES[:2], test_question)
        
        # Benchmark inference
        inference_times = []
        all_scores = []
        
        for q in TEST_QUESTIONS:
            start = time.perf_counter()
            scores = scorer.score(TEST_FRAMES, q)
            elapsed = (time.perf_counter() - start) * 1000
            inference_times.append(elapsed)
            all_scores.append(scores)
        
        avg_scores = np.mean([s.mean() for s in all_scores])
        avg_time = np.mean(inference_times)
        
        result = BenchmarkResult(
            strategy_name="clip",
            model_name=model_name,
            init_time_ms=init_time,
            avg_inference_ms=avg_time,
            total_inference_ms=sum(inference_times),
            memory_mb=memory_used,
            avg_score=avg_scores,
            score_std=np.std([s.mean() for s in all_scores]),
            min_score=np.min([s.min() for s in all_scores]),
            max_score=np.max([s.max() for s in all_scores]),
            scores=all_scores[0],
            success=True
        )
        
        print(f"  Avg inference: {avg_time:.2f} ms for {len(TEST_FRAMES)} frames")
        print(f"  Per-frame time: {avg_time/len(TEST_FRAMES):.2f} ms")
        print(f"  Avg score: {avg_scores:.3f}")
        
        clip_results.append(result)
        all_results.append(result)
        
    except Exception as e:
        print(f"Error: {e}")
        clip_results.append(BenchmarkResult(
            strategy_name="clip",
            model_name=model_name,
            success=False,
            error=str(e)
        ))
    
    # Clean up
    del scorer
    gc.collect()

Testing CLIP model: ViT-B/32
Error: module 'clip' has no attribute 'load'


NameError: name 'scorer' is not defined

In [ ]:
# Visualize CLIP scores
if clip_results and any(r.success for r in clip_results):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Score distribution per model
    ax1 = axes[0]
    for result in clip_results:
        if result.success and len(result.scores) > 0:
            ax1.plot(range(len(result.scores)), result.scores, 
                     marker='o', label=result.model_name)
    ax1.set_xlabel('Frame Index')
    ax1.set_ylabel('QFS Score')
    ax1.set_title(f'CLIP Scores: "{TEST_QUESTIONS[0][:40]}..."')
    ax1.legend()
    ax1.set_ylim(0, 1)
    
    # Timing comparison
    ax2 = axes[1]
    models = [r.model_name for r in clip_results if r.success]
    times = [r.avg_inference_ms for r in clip_results if r.success]
    colors = sns.color_palette('husl', len(models))
    bars = ax2.bar(models, times, color=colors)
    ax2.set_ylabel('Avg Inference Time (ms)')
    ax2.set_title('CLIP Model Inference Time')
    for bar, val in zip(bars, times):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                 f'{val:.1f}', ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

## **2. Multilingual CLIP Benchmark**

Testing M-CLIP for direct Vietnamese support (no translation needed).

In [ ]:
print("="*70)
print("MULTILINGUAL CLIP BENCHMARK")
print("="*70)

gc.collect()
mem_before = get_memory_mb()

try:
    init_start = time.perf_counter()
    mclip_scorer = MultilingualCLIPScoringStrategy(device="cuda")
    init_time = (time.perf_counter() - init_start) * 1000
    
    mem_after = get_memory_mb()
    memory_used = mem_after - mem_before
    
    print(f"Init time: {init_time:.2f} ms")
    print(f"Memory: {memory_used:.1f} MB")
    
    # Benchmark
    start = time.perf_counter()
    scores = mclip_scorer.score(TEST_FRAMES, TEST_QUESTIONS[0])
    inference_time = (time.perf_counter() - start) * 1000
    
    result = BenchmarkResult(
        strategy_name="mclip",
        model_name="M-CLIP/XLM-Roberta-Large-Vit-L-14",
        init_time_ms=init_time,
        avg_inference_ms=inference_time,
        memory_mb=memory_used,
        avg_score=float(np.mean(scores)),
        scores=scores,
        success=True
    )
    all_results.append(result)
    
    print(f"Inference time: {inference_time:.2f} ms")
    print(f"Avg score: {np.mean(scores):.3f}")
    
except Exception as e:
    print(f"❌ M-CLIP not available or error: {e}")
    print("   Install with: pip install multilingual-clip transformers")
    all_results.append(BenchmarkResult(
        strategy_name="mclip",
        model_name="M-CLIP",
        success=False,
        error=str(e)
    ))

gc.collect()

---
## 3. Detection Scoring Strategy Benchmark

Testing YOLO-based detection boost with different models.

In [ ]:
print("="*70)
print("DETECTION SCORING STRATEGY BENCHMARK")
print("="*70)

# YOLO models to test
YOLO_MODELS = [
    "yolo11n_unified",
    "yolo11l_unified",
]

detection_results = []

# Get target classes from question
analysis = analyzer.analyze(TEST_QUESTIONS[0])
target_classes = analysis.yolo_classes
print(f"\nTarget classes for '{TEST_QUESTIONS[0][:40]}...': {target_classes}")

for model_name in YOLO_MODELS:
    print(f"\n{'─'*60}")
    print(f"Testing YOLO model: {model_name}")
    print(f"{'─'*60}")
    
    gc.collect()
    mem_before = get_memory_mb()
    
    try:
        # Check if model exists
        model_info = ModelRegistry.get_model_info(model_name)
        if not model_info or not model_info.exists:
            print(f"  ⚠️ Model weights not found, skipping...")
            continue
        
        # Initialize
        init_start = time.perf_counter()
        yolo_model = get_model(model_name, device="cuda")
        scorer = DetectionScoringStrategy(confidence=0.25, device="cuda")
        scorer.set_model(yolo_model)
        init_time = (time.perf_counter() - init_start) * 1000
        
        mem_after = get_memory_mb()
        memory_used = mem_after - mem_before
        
        print(f"  Init time: {init_time:.2f} ms")
        print(f"  Memory: {memory_used:.1f} MB")
        
        # Warmup
        _ = scorer.score(TEST_FRAMES[:2], TEST_QUESTIONS[0], target_classes)
        
        # Benchmark
        start = time.perf_counter()
        scores = scorer.score(TEST_FRAMES, TEST_QUESTIONS[0], target_classes)
        inference_time = (time.perf_counter() - start) * 1000
        
        result = BenchmarkResult(
            strategy_name="detection",
            model_name=model_name,
            init_time_ms=init_time,
            avg_inference_ms=inference_time,
            memory_mb=memory_used,
            avg_score=float(np.mean(scores)),
            score_std=float(np.std(scores)),
            min_score=float(np.min(scores)),
            max_score=float(np.max(scores)),
            scores=scores,
            success=True
        )
        detection_results.append(result)
        all_results.append(result)
        
        print(f"  Inference time: {inference_time:.2f} ms for {len(TEST_FRAMES)} frames")
        print(f"  Per-frame time: {inference_time/len(TEST_FRAMES):.2f} ms")
        print(f"  Avg detection score: {np.mean(scores):.3f}")
        print(f"  Frames with detections: {np.sum(scores > 0)} / {len(scores)}")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        detection_results.append(BenchmarkResult(
            strategy_name="detection",
            model_name=model_name,
            success=False,
            error=str(e)
        ))
    
    gc.collect()

print(f"\n✅ Tested {len([r for r in detection_results if r.success])} YOLO models")

---
## 4. Distinctiveness Strategy Benchmark

Testing Inter-Frame Distinctiveness (IFD) scoring.

In [ ]:
print("="*70)
print("DISTINCTIVENESS STRATEGY BENCHMARK")
print("="*70)

# Test different window sizes
WINDOW_SIZES = [1, 3, 5]

ifd_results = []

for window_size in WINDOW_SIZES:
    print(f"\nTesting window_size={window_size}")
    
    gc.collect()
    
    try:
        scorer = DistinctivenessStrategy(window_size=window_size)
        
        # Benchmark
        start = time.perf_counter()
        scores = scorer.compute_from_frames(TEST_FRAMES)
        inference_time = (time.perf_counter() - start) * 1000
        
        result = BenchmarkResult(
            strategy_name="distinctiveness",
            model_name=f"window={window_size}",
            init_time_ms=0.0,
            avg_inference_ms=inference_time,
            avg_score=float(np.mean(scores)),
            score_std=float(np.std(scores)),
            min_score=float(np.min(scores)),
            max_score=float(np.max(scores)),
            scores=scores,
            success=True
        )
        ifd_results.append(result)
        all_results.append(result)
        
        print(f"  Inference time: {inference_time:.3f} ms")
        print(f"  Avg distinctiveness: {np.mean(scores):.3f}")
        print(f"  Most distinct frame: {np.argmax(scores)} (score={np.max(scores):.3f})")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")

print(f"\n✅ Tested {len(ifd_results)} window sizes")

In [ ]:
# Visualize distinctiveness scores
if ifd_results:
    fig, ax = plt.subplots(figsize=(12, 5))
    
    for result in ifd_results:
        ax.plot(range(len(result.scores)), result.scores, 
                marker='o', label=result.model_name, linewidth=2)
    
    ax.set_xlabel('Frame Index')
    ax.set_ylabel('Distinctiveness Score (IFD)')
    ax.set_title('Inter-Frame Distinctiveness by Window Size')
    ax.legend()
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---
## 5. Combined FrameScorer Benchmark

Testing the combined scorer with different weight configurations.

In [ ]:
print("="*70)
print("COMBINED FRAMESCORER BENCHMARK")
print("="*70)

# Different weight configurations to test
WEIGHT_CONFIGS = [
    {"name": "QFS Only", "alpha": 1.0, "beta": 0.0, "gamma": 0.0},
    {"name": "Balanced", "alpha": 0.5, "beta": 0.3, "gamma": 0.2},
    {"name": "Detection Focus", "alpha": 0.3, "beta": 0.5, "gamma": 0.2},
    {"name": "Diversity Focus", "alpha": 0.4, "beta": 0.2, "gamma": 0.4},
]

combined_results = []

for config in WEIGHT_CONFIGS:
    print(f"\n{'─'*60}")
    print(f"Testing: {config['name']} (α={config['alpha']}, β={config['beta']}, γ={config['gamma']})")
    print(f"{'─'*60}")
    
    gc.collect()
    mem_before = get_memory_mb()
    
    try:
        # Create scorer
        init_start = time.perf_counter()
        scorer = create_scorer(
            strategy="combined",
            clip_model="ViT-B/32",  # Use smaller model for combined tests
            device="cuda",
            use_translation=True,
            alpha=config['alpha'],
            beta=config['beta'],
            gamma=config['gamma']
        )
        
        # Set YOLO model if detection is enabled
        if config['beta'] > 0:
            try:
                yolo_model = get_model("yolo11n_unified", device="cuda")
                scorer.set_yolo_model(yolo_model)
            except:
                print("  ⚠️ YOLO model not available, detection scoring disabled")
        
        init_time = (time.perf_counter() - init_start) * 1000
        mem_after = get_memory_mb()
        
        print(f"  Init time: {init_time:.2f} ms")
        print(f"  Memory: {mem_after - mem_before:.1f} MB")
        
        # Get target classes
        analysis = analyzer.analyze(TEST_QUESTIONS[0])
        
        # Benchmark with detailed scores
        start = time.perf_counter()
        detailed_scores = scorer.score_frames(
            TEST_FRAMES, 
            TEST_QUESTIONS[0], 
            target_classes=analysis.yolo_classes,
            return_detailed=True
        )
        inference_time = (time.perf_counter() - start) * 1000
        
        final_scores = np.array([s.final_score for s in detailed_scores])
        qfs_scores = np.array([s.qfs_score for s in detailed_scores])
        det_scores = np.array([s.detection_score for s in detailed_scores])
        ifd_scores = np.array([s.ifd_score for s in detailed_scores])
        
        result = BenchmarkResult(
            strategy_name="combined",
            model_name=config['name'],
            init_time_ms=init_time,
            avg_inference_ms=inference_time,
            memory_mb=mem_after - mem_before,
            avg_score=float(np.mean(final_scores)),
            score_std=float(np.std(final_scores)),
            min_score=float(np.min(final_scores)),
            max_score=float(np.max(final_scores)),
            scores=final_scores,
            success=True
        )
        combined_results.append(result)
        all_results.append(result)
        
        print(f"  Inference time: {inference_time:.2f} ms")
        print(f"  Score breakdown:")
        print(f"    QFS mean: {np.mean(qfs_scores):.3f}")
        print(f"    Detection mean: {np.mean(det_scores):.3f}")
        print(f"    IFD mean: {np.mean(ifd_scores):.3f}")
        print(f"    Final mean: {np.mean(final_scores):.3f}")
        print(f"  Top frame: {np.argmax(final_scores)} (score={np.max(final_scores):.3f})")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        import traceback
        traceback.print_exc()
    
    gc.collect()

print(f"\n✅ Tested {len(combined_results)} weight configurations")

In [ ]:
# Visualize combined scores
if combined_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Score distribution per configuration
    ax1 = axes[0]
    for result in combined_results:
        ax1.plot(range(len(result.scores)), result.scores, 
                 marker='o', label=result.model_name, linewidth=2)
    ax1.set_xlabel('Frame Index')
    ax1.set_ylabel('Combined Score')
    ax1.set_title('Combined Scores by Weight Configuration')
    ax1.legend()
    ax1.set_ylim(0, 1)
    ax1.grid(True, alpha=0.3)
    
    # Bar chart of average scores
    ax2 = axes[1]
    names = [r.model_name for r in combined_results]
    avgs = [r.avg_score for r in combined_results]
    colors = sns.color_palette('husl', len(names))
    bars = ax2.bar(range(len(names)), avgs, color=colors)
    ax2.set_xticks(range(len(names)))
    ax2.set_xticklabels(names, rotation=45, ha='right')
    ax2.set_ylabel('Average Score')
    ax2.set_title('Average Score by Configuration')
    for bar, val in zip(bars, avgs):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                 f'{val:.3f}', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

---
## 6. Complete Results Comparison

In [ ]:
# Create summary dataframe
summary_data = []
for r in all_results:
    summary_data.append({
        "Strategy": r.strategy_name,
        "Model/Config": r.model_name,
        "Init Time (ms)": f"{r.init_time_ms:.1f}",
        "Inference (ms)": f"{r.avg_inference_ms:.1f}",
        "Memory (MB)": f"{r.memory_mb:.1f}",
        "Avg Score": f"{r.avg_score:.3f}",
        "Score Std": f"{r.score_std:.3f}" if r.score_std > 0 else "-",
        "Success": "✅" if r.success else "❌"
    })

df_summary = pd.DataFrame(summary_data)
print("\n" + "="*100)
print("COMPLETE BENCHMARK SUMMARY")
print("="*100)
display(df_summary)

In [ ]:
# Grand visualization
successful_results = [r for r in all_results if r.success]

if successful_results:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Frame Scorer Benchmark Results', fontsize=16, fontweight='bold')
    
    labels = [f"{r.strategy_name}\n({r.model_name[:15]}...)" if len(r.model_name) > 15
              else f"{r.strategy_name}\n({r.model_name})" for r in successful_results]
    colors = sns.color_palette('husl', len(successful_results))
    
    # 1. Init Time
    ax1 = axes[0, 0]
    init_times = [r.init_time_ms for r in successful_results]
    bars = ax1.bar(range(len(successful_results)), init_times, color=colors)
    ax1.set_xticks(range(len(successful_results)))
    ax1.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax1.set_ylabel('Time (ms)')
    ax1.set_title('Initialization Time')
    ax1.set_yscale('log')
    
    # 2. Inference Time
    ax2 = axes[0, 1]
    inf_times = [r.avg_inference_ms for r in successful_results]
    bars = ax2.bar(range(len(successful_results)), inf_times, color=colors)
    ax2.set_xticks(range(len(successful_results)))
    ax2.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax2.set_ylabel('Time (ms)')
    ax2.set_title('Inference Time (per batch)')
    
    # 3. Memory Usage
    ax3 = axes[1, 0]
    memory = [r.memory_mb for r in successful_results]
    bars = ax3.bar(range(len(successful_results)), memory, color=colors)
    ax3.set_xticks(range(len(successful_results)))
    ax3.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax3.set_ylabel('Memory (MB)')
    ax3.set_title('Memory Usage')
    
    # 4. Average Score
    ax4 = axes[1, 1]
    avg_scores = [r.avg_score for r in successful_results]
    bars = ax4.bar(range(len(successful_results)), avg_scores, color=colors)
    ax4.set_xticks(range(len(successful_results)))
    ax4.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax4.set_ylabel('Score')
    ax4.set_title('Average Score')
    ax4.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('../outputs/frame_scorer_benchmark.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n📁 Chart saved to: outputs/frame_scorer_benchmark.png")

---
## 7. Recommendations

In [ ]:
print("\n" + "="*80)
print("📊 RECOMMENDATIONS")
print("="*80)

if successful_results:
    # Find best performers
    fastest_init = min(successful_results, key=lambda x: x.init_time_ms if x.init_time_ms > 0 else float('inf'))
    fastest_inference = min(successful_results, key=lambda x: x.avg_inference_ms)
    highest_score = max(successful_results, key=lambda x: x.avg_score)
    lowest_memory = min([r for r in successful_results if r.memory_mb > 0], 
                        key=lambda x: x.memory_mb, default=None)
    
    print(f"\n🚀 Fastest Initialization:")
    print(f"   {fastest_init.strategy_name}/{fastest_init.model_name}: {fastest_init.init_time_ms:.2f} ms")
    
    print(f"\n⚡ Fastest Inference:")
    print(f"   {fastest_inference.strategy_name}/{fastest_inference.model_name}: {fastest_inference.avg_inference_ms:.2f} ms")
    
    print(f"\n🎯 Highest Score:")
    print(f"   {highest_score.strategy_name}/{highest_score.model_name}: {highest_score.avg_score:.3f}")
    
    if lowest_memory:
        print(f"\n💾 Lowest Memory:")
        print(f"   {lowest_memory.strategy_name}/{lowest_memory.model_name}: {lowest_memory.memory_mb:.1f} MB")

print("\n" + "─"*80)
print("\n📋 USE CASE RECOMMENDATIONS:")
print("\n  • Real-time (low latency): Use CLIP ViT-B/32 with α=1.0 (QFS only)")
print("  • Best quality: Use CLIP ViT-L/14 with balanced weights")
print("  • Object-specific queries: Enable detection scoring (β > 0)")
print("  • Reduce redundancy: Enable distinctiveness (γ > 0)")
print("  • Vietnamese support: Use translation=True (recommended) or M-CLIP")

---
## 8. Export Results

In [ ]:
# Export to CSV
export_data = []
for r in all_results:
    export_data.append({
        "strategy": r.strategy_name,
        "model": r.model_name,
        "init_time_ms": r.init_time_ms,
        "avg_inference_ms": r.avg_inference_ms,
        "memory_mb": r.memory_mb,
        "avg_score": r.avg_score,
        "score_std": r.score_std,
        "min_score": r.min_score,
        "max_score": r.max_score,
        "success": r.success,
        "error": r.error
    })

export_df = pd.DataFrame(export_data)
export_df.to_csv('../outputs/frame_scorer_benchmark.csv', index=False)

print("✅ Results exported to: outputs/frame_scorer_benchmark.csv")
print("\n" + "="*80)
print("BENCHMARK COMPLETE")
print("="*80)
print(f"\nTotal strategies tested: {len(all_results)}")
print(f"Successful: {len([r for r in all_results if r.success])}")
print(f"Failed: {len([r for r in all_results if not r.success])}")